<h1>
<hr style=" border:none; height:3px;">
<center>Main Workflow - Seamless Stable Diffusion CS</center>
<hr style=" border:none; height:3px;">
</h1>

<center><img src='https://netacad.centralesupelec.fr/img/cs.jpg' width=200></center>

<h4><center>Louis LHOTTE | Clément VERON | Edouard SEGUIER</center></h4>

# Computer Vision & specific region modification

In [2]:
from PIL import Image
import numpy as np
import cv2

<div class="alert alert-block alert-info"> The idea is to improve LEdits++ performance by splitting the prompts on foreground and background, and then reaggregate them further, enabling us to make higher-quality image. If this option was possible, this would open countless possibilities of further development (layers modification, region with clustering modification, etc...)</div>

In [4]:
def segment_foreground_background(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    foreground = cv2.bitwise_and(image, image, mask=mask)
    background = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(mask))
    return foreground, background

foreground, background = segment_foreground_background("Landscape.png")
cv2.imwrite("foreground.png", foreground)
cv2.imwrite("background.png", background)

True

# Re-Aggregation of foreground & background

<div class="alert alert-block alert-info"> This step has the advantage of being highly <b>feasible</b> as it the re-aggregation part is seemingly very fast and as long as there is no major change, the quality of the image is not degraded. It would allow artists and users to best use their creativity without breaking their previous achieved milestones (very common issue in stable diffusion or GANs).</div>

In [7]:
def reaggregate_foreground_background(foreground, background):
    return cv2.add(foreground, background)

foreground = cv2.imread("m_foreground.png")
background = cv2.imread("background.png")
reaggregated_image = reaggregate_foreground_background(foreground, background)
cv2.imwrite("reaggregated_image_v3.png", reaggregated_image)

True

However, the image lost quality so we need to upscale it again

# Upscaling

In [8]:
def upscale_image(image_path, scale_factor=2):
    image = cv2.imread(image_path)
    width = int(image.shape[1] * scale_factor)
    height = int(image.shape[0] * scale_factor)
    upscaled_image = cv2.resize(image, (width, height), interpolation=cv2.INTER_CUBIC)
    return upscaled_image

upscaled_image = upscale_image("reaggregated_image.png", scale_factor=2)
cv2.imwrite("upscaled_reaggregated_image.png", upscaled_image)

True

Not enough

# Quality infering

In [ ]:
def enhance_quality_with_modifications(original_path, modified_path):
    original = cv2.imread(original_path)
    modified = cv2.imread(modified_path)
    
    original_resized = cv2.resize(original, (modified.shape[1], modified.shape[0]), interpolation=cv2.INTER_CUBIC)
    diff_mask = cv2.absdiff(original_resized, modified)
    
    diff_mask_gray = cv2.cvtColor(diff_mask, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(diff_mask_gray, 30, 255, cv2.THRESH_BINARY)
    
    enhanced_image = cv2.addWeighted(modified, 0.7, original_resized, 0.3, 0)
    enhanced_image[binary_mask == 0] = original_resized[binary_mask == 0]
    
    return enhanced_image

enhanced_image = enhance_quality_with_modifications("original_reaggragated_version.png", "upscaled_reaggregated_image.png")
cv2.imwrite("enhanced_modified_image.png", enhanced_image)


True

# Denoising

In [12]:
def denoise_image(image_path):
    image = cv2.imread(image_path)
    denoised = cv2.fastNlMeansDenoisingColored(image, None, 10, 5, 3, 21)
    return denoised

denoised_image = denoise_image("enhanced_modified_image.png")
cv2.imwrite("denoised_enhanced_image.png", denoised_image)

True